In [ ]:
#Training 2

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tqdm import tqdm
import albumentations as A # Import albumentations

# --- 1. Configuration ---
DATASET_PATH = "/kaggle/input/traffic-dataset/traffic_wala_dataset/"
TRAIN_IMG_DIR = os.path.join(DATASET_PATH, "train/images/")
TRAIN_LBL_DIR = os.path.join(DATASET_PATH, "train/labels/")
VALID_IMG_DIR = os.path.join(DATASET_PATH, "valid/images/")
VALID_LBL_DIR = os.path.join(DATASET_PATH, "valid/labels/")

IMAGE_SIZE = (640, 640)
NUM_CLASSES = 1
BATCH_SIZE = 4
EPOCHS = 30
NUM_ANCHORS_TO_GENERATE = 9 # For K-Means

LAMBDA_COORD = 5.0
LAMBDA_NOOBJ = 0.5
LAMBDA_OBJ = 1.0
LAMBDA_CLASS = 1.0

# --- 2. K-Means for Anchor Box Generation ---
def calculate_kmeans_anchors(label_dir, num_anchors, image_size):
    """
    Reads all bounding box dimensions from the training labels and uses K-Means
    clustering to find the optimal anchor boxes.
    """
    print(f"Calculating {num_anchors} anchors using K-Means...")
    box_dims = []
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

    for label_file in tqdm(label_files, desc="Reading Bounding Boxes"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    # YOLO format: class x_center y_center width height
                    # Convert normalized w, h to pixel values
                    w = float(parts[3]) * image_size[1]
                    h = float(parts[4]) * image_size[0]
                    box_dims.append([w, h])

    if not box_dims:
        print("Warning: No bounding boxes found to calculate anchors. Using default.")
        return np.array([[10, 13], [16, 30], [33, 23], [30, 61], [62, 45], [59, 119], [116, 90], [156, 198], [373, 326]])

    box_dims = np.array(box_dims)
    kmeans = KMeans(n_clusters=num_anchors, random_state=42, n_init=10)
    kmeans.fit(box_dims)
    anchors = kmeans.cluster_centers_

    # Sort anchors by area (width * height)
    areas = anchors[:, 0] * anchors[:, 1]
    sorted_indices = np.argsort(areas)
    sorted_anchors = anchors[sorted_indices]

    print("K-Means calculated anchors (width, height):")
    print(sorted_anchors.astype(int))
    return sorted_anchors

# Calculate anchors before initializing anything else that needs them
ANCHORS = calculate_kmeans_anchors(TRAIN_LBL_DIR, NUM_ANCHORS_TO_GENERATE, IMAGE_SIZE)
ANCHORS = np.array([
    [10, 13], [16, 30], [33, 23],
    [30, 61], [62, 45], [59, 119],
    [116, 90], [156, 198], [373, 326]
])
NUM_ANCHORS = len(ANCHORS)

# --- 3. Data Augmentation Pipeline ---
# Define the augmentation pipeline using albumentations
# This is much safer for object detection as it transforms bboxes correctly
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.GaussNoise(p=0.2),
    A.Blur(blur_limit=3, p=0.1),
    # Geometric transformations that affect bounding boxes
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5,
                       border_mode=cv2.BORDER_CONSTANT, value=0),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'],
                              min_visibility=0.1, min_area=1))

import tensorflow as tf
import numpy as np
import os
import cv2

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, label_dir, batch_size, image_size, anchors, num_classes, grid_size, augment=False, shuffle=True):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.anchors = anchors
        self.num_anchors = len(anchors)
        self.num_classes = num_classes
        self.grid_size = grid_size
        self.augment = augment
        self.shuffle = shuffle
        self.image_filenames = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.indexes = np.arange(len(self.image_filenames))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_filenames = [self.image_filenames[i] for i in batch_indexes]
        X, y = self.__data_generation(batch_filenames)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_filenames):
        X = np.empty((self.batch_size, *self.image_size, 3), dtype=np.float32)
        y = np.zeros((self.batch_size, *self.grid_size, self.num_anchors, 5 + self.num_classes), dtype=np.float32)

        for i, filename in enumerate(batch_filenames):
            image_path = os.path.join(self.image_dir, filename + '.jpg')
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            bboxes = []
            class_labels = []
            label_path = os.path.join(self.label_dir, filename + '.txt')
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) == 5:
                            class_id = int(parts[0])
                            bbox = list(map(float, parts[1:]))
                            bboxes.append(bbox)
                            class_labels.append(class_id)

            # Apply augmentation (if any)
            
            if self.augment:
                transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                image = transformed['image']
                bboxes = transformed['bboxes']
                class_labels = transformed['class_labels']
            
            

            image_resized = cv2.resize(image, self.image_size)
            X[i] = image_resized / 255.0

            for j, bbox in enumerate(bboxes):
                class_id = int(class_labels[j])

                # --- FIX 1: Validate class ID ---
                if class_id < self.num_classes:
                    x_center, y_center, w, h = bbox

                    # --- FIX 2: Clamp grid coordinates ---
                    grid_x = min(int(x_center * self.grid_size[1]), self.grid_size[1] - 1)
                    grid_y = min(int(y_center * self.grid_size[0]), self.grid_size[0] - 1)

                    gt_box = np.array([0, 0, w, h])
                    best_iou = -1
                    best_anchor_idx = -1
                    for anchor_idx, anchor in enumerate(self.anchors):
                        anchor_box = np.array([0, 0, anchor[0] / self.image_size[1], anchor[1] / self.image_size[0]])
                        iou = self.iou(gt_box, anchor_box)
                        if iou > best_iou:
                            best_iou = iou
                            best_anchor_idx = anchor_idx

                    if best_anchor_idx != -1:
                        y[i, grid_y, grid_x, best_anchor_idx, 0:4] = [x_center, y_center, w, h]
                        y[i, grid_y, grid_x, best_anchor_idx, 4] = 1
                        y[i, grid_y, grid_x, best_anchor_idx, 5 + class_id] = 1  # Safe now

        return X, y

    @staticmethod
    def iou(box1, box2):
        w1, h1 = box1[2], box1[3]
        w2, h2 = box2[2], box2[3]
        inter_w = min(w1, w2)
        inter_h = min(h1, h2)
        inter_area = inter_w * inter_h
        union_area = w1 * h1 + w2 * h2 - inter_area
        return inter_area / (union_area + 1e-6)


# --- 4. Model Definition with Modified Grid Size ---
def build_model_640(target_grid_size=(18, 18)):
    base_model = tf.keras.applications.InceptionResNetV2(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    
    input_tensor = base_model.input
    # This layer in InceptionResNetV2 produces an 18x18 feature map for a 640x640 input
    backbone_output = base_model.get_layer('block8_6_ac').output
    
    print(f"Original backbone output shape: {backbone_output.shape}")

    # --- MODIFICATION TO GET 20x20 GRID ---
    # We use a Lambda layer with tf.image.resize to force the feature map to the desired size.
    # This is a flexible way to handle mismatches between backbone output and desired grid size.
    x = backbone_output
    print(f"Resized feature map shape: {x.shape}")

    x = Conv2D(512, (1, 1), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(1024, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(512, (1, 1), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    num_filters = NUM_ANCHORS * (5 + NUM_CLASSES)
    predictions_raw = Conv2D(num_filters, (1, 1), padding='same', name='detection_head')(x)
    
    model = Model(inputs=input_tensor, outputs=predictions_raw)
    return model

# Define the ideal grid size
IDEAL_GRID_SIZE = (18,18)

# Build the model
model = build_model_640(target_grid_size=IDEAL_GRID_SIZE)
model.summary()

# The grid size is now determined by our target, not the model's natural output
MODEL_GRID_H, MODEL_GRID_W = IDEAL_GRID_SIZE

# --- 5. Training Setup ---
# The yolo_loss function depends on the global ANCHORS variable.
# We need to make sure it's accessible or redefine it to accept anchors as an argument.
# For simplicity, we'll keep it as a global dependency.
def yolo_loss(y_true, y_pred):
    # This loss function remains the same as your original code
    y_pred = tf.reshape(y_pred, tf.shape(y_true))
    true_box_xy = y_true[..., 0:2]
    true_box_wh = y_true[..., 2:4]
    object_mask = y_true[..., 4:5]
    true_class_probs = y_true[..., 5:]
    
    # Grid and anchor scaling
    grid_shape = tf.shape(y_true)[1:3]
    grid_h, grid_w = grid_shape[0], grid_shape[1]
    
    anchors_tensor = tf.constant(ANCHORS, dtype=tf.float32)
    
    pred_box_xy = tf.sigmoid(y_pred[..., 0:2])
    pred_box_wh_offset = y_pred[..., 2:4]
    pred_objectness = tf.sigmoid(y_pred[..., 4:5])
    pred_class_probs = tf.nn.softmax(y_pred[..., 5:])
    
    # Decode predicted WH
    pred_box_wh_pixels = tf.exp(pred_box_wh_offset) * anchors_tensor
    
    # Scale true wh to image size for loss calculation
    true_box_wh_pixels = true_box_wh * tf.constant([IMAGE_SIZE[1], IMAGE_SIZE[0]], dtype=tf.float32)

    # Loss calculation
    box_loss_scale = 2.0 - true_box_wh[..., 0] * true_box_wh[..., 1]
    
    xy_loss = 3*LAMBDA_COORD * tf.reduce_sum(
        object_mask * box_loss_scale[..., tf.newaxis] * tf.square(true_box_xy - pred_box_xy),
        axis=[1, 2, 3, 4]
    )
    wh_loss = LAMBDA_COORD * tf.reduce_sum(
        object_mask * box_loss_scale[..., tf.newaxis] * tf.square(tf.sqrt(true_box_wh_pixels + 1e-6) - tf.sqrt(pred_box_wh_pixels + 1e-6)),
        axis=[1, 2, 3, 4]
    )
    coord_loss = xy_loss + wh_loss
    
    objectness_bce = tf.keras.losses.binary_crossentropy(object_mask, pred_objectness)
    object_mask_squeezed = tf.squeeze(object_mask, axis=-1)
    
    obj_loss = LAMBDA_OBJ * tf.reduce_sum(objectness_bce * object_mask_squeezed, axis=[1, 2, 3])
    noobj_loss = LAMBDA_NOOBJ * tf.reduce_sum(objectness_bce * (1 - object_mask_squeezed), axis=[1, 2, 3])
    objectness_loss = obj_loss + noobj_loss
    
    class_bce = tf.keras.losses.binary_crossentropy(true_class_probs, pred_class_probs)
    class_loss = LAMBDA_CLASS * tf.reduce_sum(class_bce * object_mask_squeezed, axis=[1, 2, 3])
    
    total_loss = coord_loss + objectness_loss + class_loss
    return tf.reduce_mean(total_loss)

# Instantiate generators with the new `augment` flag for training
train_generator = DataGenerator(
    TRAIN_IMG_DIR, TRAIN_LBL_DIR, BATCH_SIZE, IMAGE_SIZE, ANCHORS,
    NUM_CLASSES, (MODEL_GRID_H, MODEL_GRID_W), augment=True, shuffle=True
)
validation_generator = DataGenerator(
    VALID_IMG_DIR, VALID_LBL_DIR, BATCH_SIZE, IMAGE_SIZE, ANCHORS,
    NUM_CLASSES, (MODEL_GRID_H, MODEL_GRID_W), augment=False, shuffle=False
)

model.compile(optimizer=Adam(learning_rate=1e-4), loss=yolo_loss)

# Callbacks remain the same
checkpoint = ModelCheckpoint(
    'best_model_38x38.weights.h5', monitor='val_loss', save_best_only=True,
    save_weights_only=True, mode='min', verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

print("\n--- Starting Training ---")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint, reduce_lr, early_stop]
)

# The rest of your code for fine-tuning, plotting, and inference can remain the same.
# Just make sure to load the new weights file 'best_model_enhanced.weights.h5'.
# The decode_and_nms function will work correctly as it dynamically determines the grid size from the prediction tensor.

# --- Example of running the rest of the script ---
print("\n--- STAGE 2: Unfreezing backbone and fine-tuning ---")
model.trainable = True
model.compile(
    optimizer=Adam(learning_rate=5e-5),
    loss=yolo_loss
)
history_fine_tune = model.fit(
    train_generator,
    epochs=EPOCHS * 2, # Maybe train for more epochs in fine-tuning
    validation_data=validation_generator,
    initial_epoch=history.epoch[-1] + 1,
    callbacks=[checkpoint, reduce_lr, early_stop])

plt.figure(figsize=(12, 4))
plt.plot(history.history['loss'] + history_fine_tune.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'] + history_fine_tune.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs (Transfer + Fine-tuning)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()



In [ ]:
#16th june 2 pm 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tqdm import tqdm
import albumentations as A

# --- 1. Configuration ---
DATASET_PATH = "/kaggle/input/traffic-dataset/traffic_wala_dataset/"
TRAIN_IMG_DIR = os.path.join(DATASET_PATH, "train/images/")
TRAIN_LBL_DIR = os.path.join(DATASET_PATH, "train/labels/")
VALID_IMG_DIR = os.path.join(DATASET_PATH, "valid/images/")
VALID_LBL_DIR = os.path.join(DATASET_PATH, "valid/labels/")

IMAGE_SIZE = (640, 640)
BOX_PARAMS = 5 # x, y, w, h, confidence
BATCH_SIZE = 4
# --- CHANGE: Increased training epochs ---
EPOCHS = 50
NUM_ANCHORS_TO_GENERATE = 9

# --- CHANGE: Increased weight for object confidence loss ---
LAMBDA_COORD = 5.0
LAMBDA_NOOBJ = 0.5
LAMBDA_OBJ = 2.5 # Was 1.0. This will push the model to be more confident.

# --- 2. K-Means for Anchor Box Generation (No changes) ---
def calculate_kmeans_anchors(label_dir, num_anchors, image_size):
    print(f"Calculating {num_anchors} anchors using K-Means...")
    box_dims = []
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    for label_file in tqdm(label_files, desc="Reading Bounding Boxes"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    w = float(parts[3]) * image_size[1]
                    h = float(parts[4]) * image_size[0]
                    box_dims.append([w, h])
    if not box_dims:
        print("Warning: No bounding boxes found. Using default.")
        return np.array([[10, 13], [16, 30], [33, 23], [30, 61], [62, 45], [59, 119], [116, 90], [156, 198], [373, 326]])
    box_dims = np.array(box_dims)
    kmeans = KMeans(n_clusters=num_anchors, random_state=42, n_init=10)
    kmeans.fit(box_dims)
    anchors = kmeans.cluster_centers_
    areas = anchors[:, 0] * anchors[:, 1]
    sorted_indices = np.argsort(areas)
    sorted_anchors = anchors[sorted_indices]
    print("K-Means calculated anchors (width, height):")
    print(sorted_anchors.astype(int))
    return sorted_anchors

ANCHORS = calculate_kmeans_anchors(TRAIN_LBL_DIR, NUM_ANCHORS_TO_GENERATE, IMAGE_SIZE)
NUM_ANCHORS = len(ANCHORS)

# --- 3. Data Augmentation Pipeline (No changes) ---
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.GaussNoise(p=0.2),
    A.Blur(blur_limit=3, p=0.1),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5,
                       border_mode=cv2.BORDER_CONSTANT, value=0),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'],
                              min_visibility=0.1, min_area=1))

# --- 4. DataGenerator (No changes) ---
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, label_dir, batch_size, image_size, anchors, grid_size, augment=False, shuffle=True):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.anchors = anchors
        self.num_anchors = len(anchors)
        self.grid_size = grid_size
        self.augment = augment
        self.shuffle = shuffle
        self.image_filenames = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.indexes = np.arange(len(self.image_filenames))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_filenames = [self.image_filenames[i] for i in batch_indexes]
        X, y = self.__data_generation(batch_filenames)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_filenames):
        X = np.empty((self.batch_size, *self.image_size, 3), dtype=np.float32)
        y = np.zeros((self.batch_size, *self.grid_size, self.num_anchors, BOX_PARAMS), dtype=np.float32)
        for i, filename in enumerate(batch_filenames):
            image_path = os.path.join(self.image_dir, filename + '.jpg')
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            bboxes, class_labels = [], []
            label_path = os.path.join(self.label_dir, filename + '.txt')
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) == 5:
                            bboxes.append(list(map(float, parts[1:])))
                            class_labels.append(int(parts[0]))
            if self.augment:
                transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                image, bboxes = transformed['image'], transformed['bboxes']
            image_resized = cv2.resize(image, self.image_size)
            X[i] = image_resized / 255.0
            for bbox in bboxes:
                x_center, y_center, w, h = bbox
                grid_x = min(int(x_center * self.grid_size[1]), self.grid_size[1] - 1)
                grid_y = min(int(y_center * self.grid_size[0]), self.grid_size[0] - 1)
                gt_box = np.array([0, 0, w, h])
                best_iou, best_anchor_idx = -1, -1
                for anchor_idx, anchor in enumerate(self.anchors):
                    anchor_box = np.array([0, 0, anchor[0] / self.image_size[1], anchor[1] / self.image_size[0]])
                    iou = self.iou(gt_box, anchor_box)
                    if iou > best_iou:
                        best_iou, best_anchor_idx = iou, anchor_idx
                if best_anchor_idx != -1:
                    y[i, grid_y, grid_x, best_anchor_idx, 0:4] = [x_center, y_center, w, h]
                    y[i, grid_y, grid_x, best_anchor_idx, 4] = 1
        return X, y

    @staticmethod
    def iou(box1, box2):
        w1, h1, w2, h2 = box1[2], box1[3], box2[2], box2[3]
        inter_area = min(w1, w2) * min(h1, h2)
        union_area = w1 * h1 + w2 * h2 - inter_area
        return inter_area / (union_area + 1e-6)

# --- 5. Model Definition ---
def build_model_38x38():
    base_model = tf.keras.applications.InceptionResNetV2(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    base_model.trainable = False
    input_tensor = base_model.input
    # --- FIX: Correcting the print statement to match the layer being used ---
    backbone_output = base_model.get_layer('block17_20_ac').output 
    print(f"Using backbone output from 'block17_20_ac' with shape: {backbone_output.shape}")
    x = Conv2D(512, (1, 1), padding='same', activation=LeakyReLU(alpha=0.1))(backbone_output)
    x = Conv2D(1024, (3, 3), padding='same', activation=LeakyReLU(alpha=0.1))(x)
    num_filters = NUM_ANCHORS * BOX_PARAMS
    predictions_raw = Conv2D(num_filters, (1, 1), padding='same', name='detection_head')(x)
    model = Model(inputs=input_tensor, outputs=predictions_raw)
    return model

# --- 6. Loss Function (No changes) ---
def yolo_loss_simplified(y_true, y_pred):
    y_pred = tf.reshape(y_pred, tf.shape(y_true))
    true_box_xy = y_true[..., 0:2]
    true_box_wh = y_true[..., 2:4]
    object_mask = y_true[..., 4:5]
    pred_box_xy = tf.sigmoid(y_pred[..., 0:2])
    pred_box_wh_offset = y_pred[..., 2:4]
    pred_objectness = tf.sigmoid(y_pred[..., 4:5])
    anchors_tensor = tf.constant(ANCHORS, dtype=tf.float32)
    pred_box_wh_pixels = tf.exp(pred_box_wh_offset) * anchors_tensor
    true_box_wh_pixels = true_box_wh * tf.constant([IMAGE_SIZE[1], IMAGE_SIZE[0]], dtype=tf.float32)
    box_loss_scale = 2.0 - true_box_wh[..., 0] * true_box_wh[..., 1]
    xy_loss = LAMBDA_COORD * tf.reduce_sum(object_mask * box_loss_scale[..., tf.newaxis] * tf.square(true_box_xy - pred_box_xy), axis=[1, 2, 3, 4])
    wh_loss = LAMBDA_COORD * tf.reduce_sum(object_mask * box_loss_scale[..., tf.newaxis] * tf.square(tf.sqrt(true_box_wh_pixels + 1e-6) - tf.sqrt(pred_box_wh_pixels + 1e-6)), axis=[1, 2, 3, 4])
    coord_loss = xy_loss + wh_loss
    objectness_bce = tf.keras.losses.binary_crossentropy(object_mask, pred_objectness)
    object_mask_squeezed = tf.squeeze(object_mask, axis=-1)
    obj_loss = LAMBDA_OBJ * tf.reduce_sum(objectness_bce * object_mask_squeezed, axis=[1, 2, 3])
    noobj_loss = LAMBDA_NOOBJ * tf.reduce_sum(objectness_bce * (1 - object_mask_squeezed), axis=[1, 2, 3])
    objectness_loss = obj_loss + noobj_loss
    total_loss = coord_loss + objectness_loss
    return tf.reduce_mean(total_loss)

# --- 7. Build and Train ---
model = build_model_38x38()
model.summary()

MODEL_GRID_H, MODEL_GRID_W = model.output_shape[1:3]
print(f"Model grid size set to: ({MODEL_GRID_H}, {MODEL_GRID_W})")

train_generator = DataGenerator(TRAIN_IMG_DIR, TRAIN_LBL_DIR, BATCH_SIZE, IMAGE_SIZE, ANCHORS, (MODEL_GRID_H, MODEL_GRID_W), augment=True, shuffle=True)
validation_generator = DataGenerator(VALID_IMG_DIR, VALID_LBL_DIR, BATCH_SIZE, IMAGE_SIZE, ANCHORS, (MODEL_GRID_H, MODEL_GRID_W), augment=False, shuffle=False)

# Compile for Stage 1
model.compile(optimizer=Adam(learning_rate=1e-4), loss=yolo_loss_simplified)

# Callbacks
checkpoint = ModelCheckpoint('best_model_38x38_retrain.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1) # Increased patience
early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1) # Increased patience

print("\n--- Starting Training (Stage 1) ---")
history = model.fit(
    train_generator,
    epochs=EPOCHS, # Now 50
    validation_data=validation_generator,
    callbacks=[checkpoint, reduce_lr, early_stop]
)

# --- CHANGE: Fine-tuning stage is re-enabled and extended ---
print("\n--- STAGE 2: Unfreezing backbone and fine-tuning ---")
model.trainable = True

# It's good practice to re-compile after changing trainable status
model.compile(
    optimizer=Adam(learning_rate=1e-5), # Use a low learning rate for fine-tuning
    loss=yolo_loss_simplified
)
print("Model re-compiled for fine-tuning with a lower learning rate.")

history_fine_tune = model.fit(
    train_generator,
    epochs=EPOCHS * 2, # Fine-tune for 100 epochs
    validation_data=validation_generator,
    initial_epoch=history.epoch[-1] + 1 if history.epoch else 0,
    callbacks=[checkpoint, reduce_lr, early_stop]
)

# --- CHANGE: Robust plotting code ---
print("\n--- Generating Loss Plot ---")
combined_loss = history.history.get('loss', [])
combined_val_loss = history.history.get('val_loss', [])

# Check if fine-tuning actually ran and has a history
if hasattr(history_fine_tune, 'history') and history_fine_tune.history:
    print("Combining fine-tuning history for plotting.")
    combined_loss.extend(history_fine_tune.history.get('loss', []))
    combined_val_loss.extend(history_fine_tune.history.get('val_loss', []))
else:
    print("No fine-tuning history found to plot.")

# Plot the combined histories
plt.figure(figsize=(12, 5))
if combined_loss:
    plt.plot(combined_loss, label='Training Loss')
if combined_val_loss:
    plt.plot(combined_val_loss, label='Validation Loss')

plt.title('Model Loss Over Epochs (38x38 Grid - Retrained)')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Add a vertical line to show where fine-tuning started, if it happened
if hasattr(history_fine_tune, 'epoch') and history_fine_tune.epoch:
    start_fine_tune_epoch = history.epoch[-1] + 1 if history.epoch else 0
    plt.axvline(x=start_fine_tune_epoch, color='r', linestyle='--', label='Fine-tuning starts')

plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- PRE-REQUISITES (Ensure these match your training script) ---
# --- Configuration ---
VALID_IMG_DIR = "/kaggle/input/traffic-dataset/traffic_wala_dataset/valid/images/"
VALID_LBL_DIR = "/kaggle/input/traffic-dataset/traffic_wala_dataset/valid/labels/"
IMAGE_SIZE = (640, 640)
BOX_PARAMS = 5  # x, y, w, h, confidence
ANCHORS = np.array([
    # This should be the same ANCHORS array used during training
    [20, 29], [33,45], [48, 64],
    [63,91], [78, 140], [109,105],
    [151,201], [208,356], [396,618]
])

NUM_ANCHORS = len(ANCHORS)


# --- Model Loading ---
# Re-define the model building function exactly as it was during training
def build_model_38x38():
    base_model = tf.keras.applications.InceptionResNetV2(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    input_tensor = base_model.input
    backbone_output = base_model.get_layer('block17_20_ac').output
    x = tf.keras.layers.Conv2D(512, (1, 1), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1))(backbone_output)
    x = tf.keras.layers.Conv2D(1024, (3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1))(x)
    num_filters = NUM_ANCHORS * BOX_PARAMS
    predictions_raw = tf.keras.layers.Conv2D(num_filters, (1, 1), padding='same', name='detection_head')(x)
    model = tf.keras.models.Model(inputs=input_tensor, outputs=predictions_raw)
    return model

# 1. Build the model architecture
print("Building model architecture...")
model = build_model_38x38()

# 2. Load the trained weights
print("Loading trained weights from 'best_model_38x38.weights.h5'...")
model.load_weights('best_model_38x38.weights.h5')
print("Weights loaded successfully.")


# --- Helper Functions for Evaluation ---
# --- MODIFIED: Simplified decode_predictions for the new model output ---
def decode_predictions_simplified(raw_preds):
    """
    Decodes the raw output of the simplified YOLO model (no class prediction).
    """
    grid_h, grid_w = raw_preds.shape[1:3]
    
    # Reshape to (batch, grid_h, grid_w, num_anchors, 5)
    predictions = tf.reshape(raw_preds, (tf.shape(raw_preds)[0], grid_h, grid_w, NUM_ANCHORS, BOX_PARAMS))

    # Get individual components
    pred_xy = tf.sigmoid(predictions[..., 0:2])
    pred_wh_offset = predictions[..., 2:4]
    pred_conf = tf.sigmoid(predictions[..., 4:5])

    # Create a grid for calculating absolute coordinates
    grid = tf.meshgrid(tf.range(grid_w, dtype=tf.float32), tf.range(grid_h, dtype=tf.float32))
    grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)

    # Decode box coordinates
    box_xy_grid = pred_xy + grid
    box_xy_normalized = box_xy_grid / tf.constant([grid_w, grid_h], dtype=tf.float32)
    
    anchors_tensor = tf.constant(ANCHORS, dtype=tf.float32)
    box_wh_pixels = tf.exp(pred_wh_offset) * anchors_tensor
    box_wh_normalized = box_wh_pixels / tf.constant([IMAGE_SIZE[1], IMAGE_SIZE[0]], dtype=tf.float32)

    # Convert from (center_x, center_y, w, h) to (x1, y1, x2, y2)
    box_x1y1 = box_xy_normalized - (box_wh_normalized / 2.0)
    box_x2y2 = box_xy_normalized + (box_wh_normalized / 2.0)
    boxes = tf.concat([box_x1y1, box_x2y2], axis=-1)

    # Flatten the grid/anchor dimensions to get a list of predictions per image
    boxes_flat = tf.reshape(boxes, (tf.shape(boxes)[0], -1, 4))
    scores_flat = tf.reshape(pred_conf, (tf.shape(pred_conf)[0], -1))
    
    # Since there's only one class, the class ID is always 0
    classes_flat = tf.zeros_like(scores_flat, dtype=tf.int32)
    
    return boxes_flat, scores_flat, classes_flat


def calculate_iou(box1, box2):
    # This function remains the same
    x1_inter = np.maximum(box1[:, 0], box2[:, 0])
    y1_inter = np.maximum(box1[:, 1], box2[:, 1])
    x2_inter = np.minimum(box1[:, 2], box2[:, 2])
    y2_inter = np.minimum(box1[:, 3], box2[:, 3])
    inter_area = np.maximum(0.0, x2_inter - x1_inter) * np.maximum(0.0, y2_inter - y1_inter)
    box1_area = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
    box2_area = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / (union_area + 1e-6)

# --- Main Evaluation Logic ---
def evaluate_model(model, img_dir, lbl_dir, conf_thresh=0.1, nms_thresh=0.5, iou_threshold=0.5):
    all_detections, all_ground_truths = [], []
    validation_files = [os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith('.jpg')]

    print(f"\nRunning evaluation on {len(validation_files)} validation images...")
    
    for i, filename in enumerate(tqdm(validation_files, desc="Evaluating")):
        # Get Predictions
        image_path = os.path.join(img_dir, filename + '.jpg')
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, IMAGE_SIZE)
        input_data = np.expand_dims(image_resized / 255.0, axis=0)
        raw_preds = model.predict(input_data, verbose=0)
        
        # Use the new simplified decoder
        pred_boxes, pred_scores, pred_classes = decode_predictions_simplified(raw_preds)
        
        selected_indices = tf.image.non_max_suppression(
            boxes=pred_boxes[0],
            scores=pred_scores[0],
            max_output_size=50,
            iou_threshold=nms_thresh,
            score_threshold=conf_thresh
        )
        
        final_boxes = tf.gather(pred_boxes[0], selected_indices).numpy()
        final_scores = tf.gather(pred_scores[0], selected_indices).numpy()
        final_classes = tf.gather(pred_classes[0], selected_indices).numpy()
        
        for box, score, cls in zip(final_boxes, final_scores, final_classes):
            all_detections.append([i, int(cls), float(score), *box])
            
        # Get Ground Truths
        label_path = os.path.join(lbl_dir, filename + '.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_id = int(parts[0]) # This will always be 0
                    x_c, y_c, w, h = map(float, parts[1:])
                    x1, y1 = x_c - w / 2, y_c - h / 2
                    x2, y2 = x_c + w / 2, y_c + h / 2
                    all_ground_truths.append([i, class_id, 0, x1, y1, x2, y2])
                    
    # Calculate mAP
    # Since we only have one class, mAP is just the AP of that class.
    detections_c = [d for d in all_detections if d[1] == 0]
    ground_truths_c = [gt for gt in all_ground_truths if gt[1] == 0]

    ap = 0.0
    if len(ground_truths_c) > 0:
        detections_c.sort(key=lambda x: x[2], reverse=True)
        num_gt_boxes = len(ground_truths_c)
        true_positives = np.zeros(len(detections_c))
        false_positives = np.zeros(len(detections_c))
        gt_matched = {}

        for det_idx, detection in enumerate(detections_c):
            img_idx = detection[0]
            gts_in_image = [gt for gt in ground_truths_c if gt[0] == img_idx]
            if len(gts_in_image) == 0:
                false_positives[det_idx] = 1
                continue
            det_box = np.array([detection[3:]])
            gt_boxes = np.array([gt[3:] for gt in gts_in_image])
            ious = calculate_iou(det_box, gt_boxes)
            best_iou_idx = np.argmax(ious)
            if ious[best_iou_idx] >= iou_threshold:
                if gt_matched.get(img_idx) is None: gt_matched[img_idx] = []
                if best_iou_idx not in gt_matched[img_idx]:
                    true_positives[det_idx] = 1
                    gt_matched[img_idx].append(best_iou_idx)
                else:
                    false_positives[det_idx] = 1
            else:
                false_positives[det_idx] = 1
        tp_cumsum = np.cumsum(true_positives)
        fp_cumsum = np.cumsum(false_positives)
        recalls = tp_cumsum / (num_gt_boxes + 1e-6)
        precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
        precisions = np.concatenate(([1.0], precisions))
        recalls = np.concatenate(([0.0], recalls))
        for i in range(len(precisions) - 2, -1, -1):
            precisions[i] = np.maximum(precisions[i], precisions[i+1])
        recall_diff = recalls[1:] - recalls[:-1]
        ap = np.sum(recall_diff * precisions[1:])
    
    print("\n--- Evaluation Metrics ---")
    print(f"Confidence Threshold: {conf_thresh}")
    print(f"NMS Threshold: {nms_thresh}")
    print(f"IoU Threshold for mAP: {iou_threshold}")
    print("--------------------------")
    print(f"Average Precision (AP) for Vehicle: {ap:.4f}")
    
    return ap

def visualize_evaluation(model, img_dir, lbl_dir, num_images=5, conf_thresh=0.5, nms_thresh=0.1):
    # This visualization function is adapted for the new model
    validation_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
    if len(validation_files) < num_images: num_images = len(validation_files)
    sample_files = np.random.choice(validation_files, num_images, replace=False)

    for filename in sample_files:
        image_path = os.path.join(img_dir, filename)
        image = cv2.imread(image_path)
        original_h, original_w, _ = image.shape
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, IMAGE_SIZE)
        input_data = np.expand_dims(image_resized / 255.0, axis=0)
        raw_preds = model.predict(input_data, verbose=0)
        
        pred_boxes, pred_scores, _ = decode_predictions_simplified(raw_preds)
        selected_indices = tf.image.non_max_suppression(
            boxes=pred_boxes[0], scores=pred_scores[0], max_output_size=50,
            iou_threshold=nms_thresh, score_threshold=conf_thresh
        )
        final_boxes = tf.gather(pred_boxes[0], selected_indices).numpy()
        final_scores = tf.gather(pred_scores[0], selected_indices).numpy()

        for box, score in zip(final_boxes, final_scores):
            x1, y1, x2, y2 = box
            x1, y1 = int(x1 * original_w), int(y1 * original_h)
            x2, y2 = int(x2 * original_w), int(y2 * original_h)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"Vehicle: {score:.2f}"
            cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Draw Ground Truth Boxes
        label_path = os.path.join(lbl_dir, os.path.splitext(filename)[0] + '.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    _, x_c, y_c, w, h = map(float, line.strip().split())
                    x1, y1 = int((x_c - w/2) * original_w), int((y_c - h/2) * original_h)
                    x2, y2 = int((x_c + w/2) * original_w), int((y_c + h/2) * original_h)
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f"Evaluation: {filename}")
        plt.axis('off')
        plt.show()

# --- Run the Evaluation ---
if __name__ == '__main__':
    # Calculate AP@0.5
    evaluate_model(model, VALID_IMG_DIR, VALID_LBL_DIR, iou_threshold=0.2, conf_thresh=0.1)
    
    # Visualize some results with a slightly higher confidence for cleaner plots
    print("\nDisplaying sample evaluation images (Green=Prediction, Blue=Ground Truth)...")
    visualize_evaluation(model, VALID_IMG_DIR, VALID_LBL_DIR, num_images=5, conf_thresh=0.5)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- PRE-REQUISITES (Ensure these are defined from your training script) ---
# Make sure these variables match your training setup EXACTLY.

# --- Configuration ---
VALID_IMG_DIR = "/kaggle/input/traffic-dataset/traffic_wala_dataset/valid/images/"
VALID_LBL_DIR = "/kaggle/input/traffic-dataset/traffic_wala_dataset/valid/labels/"
IMAGE_SIZE = (640, 640)
NUM_CLASSES = 1
NUM_ANCHORS = 9
confidence_thresh = 0.5
nms_thresh = 0.5
iou_threshold = 0.5

# --- Model Loading ---
# You need the model building function to create the architecture first
# Make sure this function is identical to the one you used for training.
def build_model_640():
    
        base_model = tf.keras.applications.InceptionResNetV2(
            input_shape=(*IMAGE_SIZE, 3),
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False
        
        input_tensor = base_model.input
        # This layer in InceptionResNetV2 produces an 18x18 feature map for a 640x640 input
        backbone_output = base_model.get_layer('block8_6_ac').output
        
        print(f"Original backbone output shape: {backbone_output.shape}")
    
        # --- MODIFICATION TO GET 20x20 GRID ---
        # We use a Lambda layer with tf.image.resize to force the feature map to the desired size.
        # This is a flexible way to handle mismatches between backbone output and desired grid size.
        x = backbone_output
        print(f"Resized feature map shape: {x.shape}")
    
        x = Conv2D(512, (1, 1), padding='same')(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = Conv2D(1024, (3, 3), padding='same')(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = Conv2D(512, (1, 1), padding='same')(x)
        x = LeakyReLU(alpha=0.1)(x)
        
        num_filters = NUM_ANCHORS * (5 + NUM_CLASSES)
        predictions_raw = Conv2D(num_filters, (1, 1), padding='same', name='detection_head')(x)
        
        model = Model(inputs=input_tensor, outputs=predictions_raw)
        return model
# 1. Build the model architecture
print("Building model architecture...")
model = build_model_640()

# 2. Load the trained weights
print("Loading trained weights from 'best_model_enhanced_1.weights.h5'...")
model.load_weights('best_model_enhanced_1.weights.h5')
print("Weights loaded successfully.")


# --- Helper Functions for Evaluation ---

def decode_predictions(raw_preds, confidence_thresh=0.7): # Lower the initial threshold here
    """
    Decodes the raw output of the YOLO model and flattens the output
    to be compatible with tf.image.non_max_suppression.
    """
    grid_h, grid_w = raw_preds.shape[1:3]
    
    # Reshape to (batch_size, grid_h, grid_w, num_anchors, 5 + num_classes)
    predictions = tf.reshape(raw_preds, (tf.shape(raw_preds)[0], grid_h, grid_w, NUM_ANCHORS, 5 + NUM_CLASSES))

    # Get individual components
    pred_xy = tf.sigmoid(predictions[..., 0:2])
    pred_wh_offset = predictions[..., 2:4]
    pred_conf = tf.sigmoid(predictions[..., 4:5])
    pred_class_probs = tf.nn.softmax(predictions[..., 5:])

    # Create a grid for calculating absolute coordinates
    grid = tf.meshgrid(tf.range(grid_w, dtype=tf.float32), tf.range(grid_h, dtype=tf.float32))
    grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)

    # Decode box coordinates
    box_xy_grid = pred_xy + grid
    box_xy_normalized = box_xy_grid / tf.constant([grid_w, grid_h], dtype=tf.float32)
    
    anchors_tensor = tf.constant(ANCHORS, dtype=tf.float32)
    box_wh_pixels = tf.exp(pred_wh_offset) * anchors_tensor
    box_wh_normalized = box_wh_pixels / tf.constant([IMAGE_SIZE[1], IMAGE_SIZE[0]], dtype=tf.float32)

    box_x1y1 = box_xy_normalized - (box_wh_normalized / 2.0)
    box_x2y2 = box_xy_normalized + (box_wh_normalized / 2.0)
    boxes = tf.concat([box_x1y1, box_x2y2], axis=-1)

    # Calculate final scores and get class predictions
    final_scores = pred_conf * pred_class_probs
    class_ids = tf.argmax(final_scores, axis=-1)
    max_scores = tf.reduce_max(final_scores, axis=-1)

    # --- THE FIX IS HERE ---
    # We flatten the grid and anchor dimensions to get a list of boxes per image.
    # The new shape will be (batch_size, num_all_boxes, 4)
    boxes_flat = tf.reshape(boxes, (tf.shape(boxes)[0], -1, 4))
    
    # Do the same for scores and classes
    # New shape: (batch_size, num_all_boxes)
    scores_flat = tf.reshape(max_scores, (tf.shape(max_scores)[0], -1))
    classes_flat = tf.reshape(class_ids, (tf.shape(class_ids)[0], -1))

    # Optional: Filter out boxes with very low scores early to speed up NMS
    # This is not strictly necessary but is good practice.
    score_mask = scores_flat > confidence_thresh
    
    final_boxes = tf.boolean_mask(boxes_flat, score_mask)
    final_scores = tf.boolean_mask(scores_flat, score_mask)
    final_classes = tf.boolean_mask(classes_flat, score_mask)
    
    # NMS expects a single tensor for each, not batched, so we remove the batch dim
    # This assumes we are always processing one image at a time in evaluation
    # To handle batches, you would loop or use tf.map_fn
    
    # We will return the flattened tensors and let the main loop handle NMS
    return boxes_flat, scores_flat, classes_flat

def calculate_iou(box1, box2):
    """Calculates IoU for two sets of boxes.
    box format: [x1, y1, x2, y2]
    """
    x1_inter = np.maximum(box1[:, 0], box2[:, 0])
    y1_inter = np.maximum(box1[:, 1], box2[:, 1])
    x2_inter = np.minimum(box1[:, 2], box2[:, 2])
    y2_inter = np.minimum(box1[:, 3], box2[:, 3])

    inter_area = np.maximum(0.0, x2_inter - x1_inter) * np.maximum(0.0, y2_inter - y1_inter)

    box1_area = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
    box2_area = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / (union_area + 1e-6)

# --- Main Evaluation Logic ---

def evaluate_model(model, img_dir, lbl_dir, conf_thresh=0.5, nms_thresh=0.5, iou_threshold=0.5):
    """
    Main function to calculate mAP and other metrics.
    """
    all_detections = []
    all_ground_truths = []
    
    validation_files = [os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith('.jpg')]

    print(f"\nRunning evaluation on {len(validation_files)} validation images...")
    
    for i, filename in enumerate(tqdm(validation_files)):
        # --- 1. Get Predictions ---
        image_path = os.path.join(img_dir, filename + '.jpg')
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, IMAGE_SIZE)
        input_data = np.expand_dims(image_resized / 255.0, axis=0)
        
        raw_preds = model.predict(input_data, verbose=0)
        
        # Decode predictions and apply NMS
        pred_boxes, pred_scores, pred_classes = decode_predictions(raw_preds, confidence_thresh)
        
        # Non-Max Suppression
        selected_indices = tf.image.non_max_suppression(
            boxes=pred_boxes[0],
            scores=pred_scores[0],
            max_output_size=50,
            iou_threshold=nms_thresh
        )
        
        final_boxes = tf.gather(pred_boxes[0], selected_indices).numpy()
        final_scores = tf.gather(pred_scores[0], selected_indices).numpy()
        final_classes = tf.gather(pred_classes[0], selected_indices).numpy()
        
        for box, score, cls in zip(final_boxes, final_scores, final_classes):
            # Store as [image_idx, class_id, confidence, x1, y1, x2, y2]
            all_detections.append([i, int(cls), float(score), *box])
            
        # --- 2. Get Ground Truths ---
        label_path = os.path.join(lbl_dir, filename + '.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    x_c, y_c, w, h = map(float, parts[1:])
                    x1 = x_c - w / 2
                    y1 = y_c - h / 2
                    x2 = x_c + w / 2
                    y2 = y_c + h / 2
                    # Store as [image_idx, class_id, 0 (not a detection), x1, y1, x2, y2]
                    all_ground_truths.append([i, class_id, 0, x1, y1, x2, y2])
                    
    # --- 3. Calculate mAP ---
    average_precisions = {}
    
    # Process one class at a time (here we only have one class: 0)
    for class_id in range(NUM_CLASSES):
        # Get all detections and ground truths for this class
        detections_c = [d for d in all_detections if d[1] == class_id]
        ground_truths_c = [gt for gt in all_ground_truths if gt[1] == class_id]

        if len(ground_truths_c) == 0:
            average_precisions[class_id] = 1.0 if len(detections_c) == 0 else 0.0
            continue
            
        # Sort detections by confidence score (high to low)
        detections_c.sort(key=lambda x: x[2], reverse=True)
        
        num_gt_boxes = len(ground_truths_c)
        true_positives = np.zeros(len(detections_c))
        false_positives = np.zeros(len(detections_c))
        
        # Keep track of which ground truth boxes have been matched
        gt_matched = {} # key: image_idx, value: list of matched gt box indices

        for det_idx, detection in enumerate(detections_c):
            img_idx = detection[0]
            
            # Get ground truths for the same image
            gts_in_image = [gt for gt in ground_truths_c if gt[0] == img_idx]
            
            if len(gts_in_image) == 0:
                false_positives[det_idx] = 1
                continue
            
            det_box = np.array([detection[3:]])
            gt_boxes = np.array([gt[3:] for gt in gts_in_image])
            
            ious = calculate_iou(det_box, gt_boxes)
            best_iou_idx = np.argmax(ious)
            
            if ious[best_iou_idx] >= iou_threshold:
                # Check if this ground truth box was already matched
                if gt_matched.get(img_idx) is None:
                    gt_matched[img_idx] = []
                
                if best_iou_idx not in gt_matched[img_idx]:
                    true_positives[det_idx] = 1
                    gt_matched[img_idx].append(best_iou_idx)
                else: # Matched a GT box that was already used
                    false_positives[det_idx] = 1
            else: # Did not meet IoU threshold
                false_positives[det_idx] = 1

        # Calculate precision and recall
        tp_cumsum = np.cumsum(true_positives)
        fp_cumsum = np.cumsum(false_positives)
        
        recalls = tp_cumsum / (num_gt_boxes + 1e-6)
        precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
        
        # AP is the area under the precision-recall curve
        precisions = np.concatenate(([1.0], precisions))
        recalls = np.concatenate(([0.0], recalls))
        
        # Replace each precision value with the maximum precision value to its right
        for i in range(len(precisions) - 2, -1, -1):
            precisions[i] = np.maximum(precisions[i], precisions[i+1])
            
        # Calculate the area
        recall_diff = recalls[1:] - recalls[:-1]
        ap = np.sum(recall_diff * precisions[1:])
        average_precisions[class_id] = ap

    mAP = np.mean(list(average_precisions.values()))
    
    print("\n--- Evaluation Metrics ---")
    print(f"Confidence Threshold: {conf_thresh}")
    print(f"NMS Threshold: {nms_thresh}")
    print(f"IoU Threshold for mAP: {iou_threshold}")
    print("--------------------------")
    for class_id, ap in average_precisions.items():
        print(f"AP for class {class_id} (Vehicle): {ap:.4f}")
    print(f"\nMean Average Precision (mAP)@{iou_threshold}: {mAP:.4f}")
    
    return mAP, average_precisions

def visualize_evaluation(model, img_dir, lbl_dir, num_images=5, conf_thresh=0.25, nms_thresh=0.5):
    """
    Draws ground truth boxes and prediction boxes on a few sample images.
    INCLUDES THE CRUCIAL SCORE THRESHOLD.
    """
    validation_files = [os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith('.jpg')]
    if len(validation_files) < num_images:
        print(f"Warning: Requesting {num_images} images to visualize, but only {len(validation_files)} are available.")
        num_images = len(validation_files)
        
    sample_files = np.random.choice(validation_files, num_images, replace=False)

    for filename in sample_files:
        image_path = os.path.join(img_dir, filename + '.jpg')
        image = cv2.imread(image_path)
        original_h, original_w, _ = image.shape
        
        # Prepare image for model
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, IMAGE_SIZE)
        input_data = np.expand_dims(image_resized / 255.0, axis=0)

        # Get Predictions using the corrected decode function
        pred_boxes, pred_scores, pred_classes = decode_predictions(model.predict(input_data, verbose=0))

        # --- THE FIX IS HERE ---
        # Add the `score_threshold` parameter to the NMS call.
        # This will first discard all boxes with a score < conf_thresh, THEN run NMS.
        selected_indices = tf.image.non_max_suppression(
            boxes=pred_boxes[0],
            scores=pred_scores[0],
            max_output_size=50,
            iou_threshold=nms_thresh,
            score_threshold=conf_thresh  # <-- THIS IS THE CRUCIAL FIX
        )
        
        final_boxes = tf.gather(pred_boxes[0], selected_indices).numpy()
        final_scores = tf.gather(pred_scores[0], selected_indices).numpy()

        # Draw Prediction Boxes (in green)
        for box, score in zip(final_boxes, final_scores):
            x1, y1, x2, y2 = box
            x1 = int(x1 * original_w)
            y1 = int(y1 * original_h)
            x2 = int(x2 * original_w)
            y2 = int(y2 * original_h)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"Pred: {score:.2f}"
            cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Draw Ground Truth Boxes (in blue)
        label_path = os.path.join(lbl_dir, filename + '.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    _, x_c, y_c, w, h = map(float, line.strip().split())
                    x1 = int((x_c - w/2) * original_w)
                    y1 = int((y_c - h/2) * original_h)
                    x2 = int((x_c + w/2) * original_w)
                    y2 = int((y_c + h/2) * original_h)
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(image, 'Ground Truth', (x1, y2+15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        
        plt.figure(figsize=(12, 12))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f"Evaluation: {filename}.jpg")
        plt.axis('off')
        plt.show()
# --- Run the Evaluation ---
if __name__ == '__main__':
    # Calculate mAP@0.5 (the standard PASCAL VOC metric)
    evaluate_model(model, VALID_IMG_DIR, VALID_LBL_DIR, iou_threshold=0.2)
    
    # You could also calculate mAP for a stricter IoU
    # evaluate_model(model, VALID_IMG_DIR, VALID_LBL_DIR, iou_threshold=0.75)
    
    # Visualize some results
    print("\nDisplaying sample evaluation images (Green=Prediction, Blue=Ground Truth)...")
    visualize_evaluation(model, VALID_IMG_DIR, VALID_LBL_DIR, num_images=5)
"""     """



In [ ]:
#Testing on best_model_enhanced_1.weights.h5

In [ ]:
import time
from tqdm import tqdm
print("\n--- Loading best weights for inference ---")
model.load_weights('best_model_enhanced_1.weights.h5')

def decode_and_nms(predictions, confidence_thresh=0.5, nms_thresh=0.2):
    grid_h, grid_w = predictions.shape[1:3]
    predictions = tf.reshape(predictions, (tf.shape(predictions)[0], grid_h, grid_w, NUM_ANCHORS, 5 + NUM_CLASSES))
    pred_xy = tf.sigmoid(predictions[..., 0:2])
    pred_wh_offset = predictions[..., 2:4]
    pred_conf = tf.sigmoid(predictions[..., 4:5])
    pred_class = tf.nn.softmax(predictions[..., 5:])
    grid = tf.meshgrid(tf.range(grid_w), tf.range(grid_h))
    grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)
    grid = tf.cast(grid, tf.float32)
    box_center_grid = pred_xy + grid
    box_wh_pixels = tf.exp(pred_wh_offset) * ANCHORS
    box_xy_normalized = box_center_grid / tf.constant([grid_w, grid_h], dtype=tf.float32)
    box_wh_normalized = box_wh_pixels / tf.constant([IMAGE_SIZE[0], IMAGE_SIZE[1]], dtype=tf.float32)
    box_x1y1 = box_xy_normalized - (box_wh_normalized / 2)
    box_x2y2 = box_xy_normalized + (box_wh_normalized / 2)
    boxes_tensor = tf.concat([box_x1y1, box_x2y2], axis=-1)
    final_scores = pred_conf * pred_class
    max_scores = tf.reduce_max(final_scores, axis=-1)
    mask = max_scores >= confidence_thresh
    boxes_filtered = tf.boolean_mask(boxes_tensor, mask)
    scores_filtered = tf.boolean_mask(max_scores, mask)
    selected_indices = tf.image.non_max_suppression(
        boxes_filtered, scores_filtered, max_output_size=50, iou_threshold=nms_thresh
    )
    final_boxes = tf.gather(boxes_filtered, selected_indices)
    final_scores = tf.gather(scores_filtered, selected_indices)
    return final_boxes.numpy(), final_scores.numpy()    

def run_inference_and_draw(model, image_path, confidence_thresh=0.6, nms_thresh=0.2):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image at {image_path}")
        return None
    original_h, original_w, _ = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, IMAGE_SIZE)
    input_data = image_resized / 255.0
    input_data = np.expand_dims(input_data, axis=0)
    raw_preds = model.predict(input_data, verbose=0)
    boxes, scores = decode_and_nms(raw_preds, confidence_thresh=confidence_thresh, nms_thresh=nms_thresh)
    print("\n--- Decoded Boxes (normalized coordinates) ---")
    print(boxes)
    print("--------------------------------------------")
    print(f"Found {len(boxes)} vehicles in the image.")
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(1, x2), min(1, y2)
        x1, y1 = int(x1 * original_w), int(y1 * original_h)
        x2, y2 = int(x2 * original_w), int(y2 * original_h)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"Vehicle: {scores[i]:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)
    return image
for i in range(90):
    
    try:
        validation_image_files = os.listdir(VALID_IMG_DIR)
        if validation_image_files:
            sample_image_name = validation_image_files[i]
            sample_image_path = os.path.join(VALID_IMG_DIR, sample_image_name)
            print(f"\n--- Running visualization on: {sample_image_path} ---")
            result_image = run_inference_and_draw(model, sample_image_path)
            if result_image is not None:
                plt.figure(figsize=(12, 12))
                plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.title("Single Image Inference Result")
                plt.show()
        else:
            print("No images found in validation directory to run visualization.")
    except Exception as e:
        print(f"Could not run visualization. Error: {e}")
